upload the zip file and unzip it

In [ ]:
!unzip -q 2024-09-23-preprocessed-dataset.zip -d POP909

import statements and groundwork

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, TimeDistributed, SimpleRNN, Dropout, Attention, Concatenate, Reshape
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.callbacks import EarlyStopping
import pickle

function definitions

In [ ]:
# Define the fixed vocabulary
VOCABULARY = ["-", "_"] + [str(i) for i in range(128)]

def create_tokenizer():
    tokenizer = Tokenizer(char_level=False, filters='')
    tokenizer.fit_on_texts([VOCABULARY])
    return tokenizer

def load_and_preprocess_data(file_paths, tokenizer, sequence_length=32):
    melodies = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            melody = file.read().strip().split()
            melodies.append(melody)

    # Convert melodies to sequences
    sequences = tokenizer.texts_to_sequences(melodies)

    # Prepare input-output pairs
    X, y = [], []
    for sequence in sequences:
        for i in range(0, len(sequence) - sequence_length*2 + 1, sequence_length):
            X.append(sequence[i:i+sequence_length])
            y.append(sequence[i+sequence_length:i+sequence_length*2])
    X = np.array(X)
    y = np.array(y)
    return X, y

# Un Nouvel Nouvel Essai

def create_model(vocab_size, sequence_length):
    model = Sequential([
        Embedding(vocab_size, 128),
        LSTM(512, return_sequences=True),
        Dropout(0.2),
        LSTM(512, return_sequences=True),
        Dropout(0.2),
        LSTM(512, return_sequences=True),
        TimeDistributed(Dense(512, activation='relu')),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

def generate_melody(model, tokenizer, input_sequence, temperature=1.0):
    input_seq = tokenizer.texts_to_sequences([input_sequence])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=32, truncating='pre')

    prediction = model.predict(input_seq, verbose=0)[0]

    # Apply temperature
    prediction = np.log(prediction) / temperature
    exp_preds = np.exp(prediction)
    prediction = exp_preds / np.sum(exp_preds, axis=-1, keepdims=True)

    output_seq = []
    for step in range(32):
        probas = prediction[step]
        choice = np.random.choice(len(probas), p=probas)
        output_seq.append(choice)

    output_melody = ' '.join([tokenizer.index_word.get(token, '') for token in output_seq])
    return output_melody

def convert_to_tflite(model_path, tflite_path):
    model = load_model(model_path)
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    tflite_model = converter.convert()

    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)

def generate_melody_tflite(interpreter, tokenizer, input_sequence, temperature=1.0):
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_seq = tokenizer.texts_to_sequences([input_sequence])[0]
    input_seq = tf.keras.preprocessing.sequence.pad_sequences([input_seq], maxlen=32, truncating='pre')
    input_seq = np.array(input_seq, dtype=np.float32)  # Convert input to float32

    interpreter.set_tensor(input_details[0]['index'], input_seq)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])[0]

    # Apply temperature
    prediction = np.log(prediction) / temperature
    exp_preds = np.exp(prediction)
    prediction = exp_preds / np.sum(exp_preds, axis=-1, keepdims=True)

    output_seq = []
    for step in range(32):
        probas = prediction[step]
        choice = np.random.choice(len(probas), p=probas)
        output_seq.append(choice)

    output_melody = ' '.join([tokenizer.index_word.get(token, '') for token in output_seq])
    return output_melody

load data for training

In [ ]:
# Usage
file_paths = [f'/content/POP909/{i:06d}_melody.txt' for i in range(1, 10909)]
tokenizer = create_tokenizer()
X, y = load_and_preprocess_data(file_paths, tokenizer)
# Reshape y to be 3D
y = y[:, :, np.newaxis]

train the model

In [ ]:
# Create and train the model
vocab_size = len(VOCABULARY)
model = create_model(vocab_size, sequence_length=32)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X, y, batch_size=32, epochs=100, validation_split=0.1, callbacks=[early_stopping])

save model and tokenizer (fuck tokenizers)

In [ ]:
# Save the model and tokenizer
model.save('20240923_generative_delay.h5')
with open('20240923_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

test: generate melody

In [ ]:
# Generate a melody using the .h5 model
input_melody = "59 - 59 - 62 - 62 - 59 - 59 - 62 - 62 - 66 - 66 - 62 - 62 - 66 - 66 - 69 - 69 -"
generated_melody = generate_melody(model, tokenizer, input_melody, temperature=0.7)
print("Generated melody:", generated_melody)
